# IT Security - Sheet 7 "Firewalls, Intrusion Detection, and Access Control"

**Total achievable points: 20**

**Released: 30.01.2025**

**Submission Deadline: 06.02.2025 08:30**

Note the **earlier** submission deadline as the solution to this sheet will be discussed in the global exercise on the 30th of January.

---
Group: FILL IN

Names and matriculation numbers of **ALL** team members: FILL IN 

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## 0. Setup

Like the last exercise, this exercise requires the [scapy](https://scapy.readthedocs.io/en/latest/index.html) Python package. The installation depends on your development environment, but you can probably install it by uncommenting the code in the cell below. Note that there we have included a scapy cheat-sheet, which should provide you a good starting point for any questions w.r.t. scapy in this exercise.

In [ ]:
### PLAYGROUND
#import sys
#!{sys.executable} -m pip install scapy

---

You continue your employment at NoodleFort, the everything-security manifacturer. And, as it seems, you'll have to cover quite a lot of security-related topics this week.  

## 1. Under Attack (5 points)

One of NoodleFort's customers is currently the victim of a massive DoS attack. Concretely, the attacker is employing a large-scale syn flood attack. You remember having heard about a protective measure against such attacks, called TCP cookies.

##### a) (2 points)

**Look up** TCP SYN cookies. **Explain briefly** what they do and how they protect against TCP SYN flood attacks.

YOUR ANSWER HERE

##### b) (1 point)

While enabling TCP SYN cookies prevented the attack from filling up the connection table, the attack is so massive that the customer's bandwidth is fully saturated and legitimate users no longer can reach the website. The sales people at NoodleFort immediately notice an oppertunity to sell more powerful network equipment, and ask you to calculate how powerful the network equipment must be to handle the attack.

---

**Calculate** how much bandwidth a SYN flood sending 1000000000 ($10^9$) SYN packets per second consumes. You can assume that each relevant TCP packet (SYN, SYN-ACK, RST) is 74 bytes large, including IP header and Ethernet overhead. You can further assume that the attacker is spoofing the source IP addresses and that real devices corresponding to the spoofed source addresses always respond with a RST packet.

YOUR ANSWER HERE

##### c) (1 point)

The equipment needed to handle these kinds of throughputs is not cheap, and the customer therefore wants to be absolutely sure that simple changes for the attacker do not result in even larger bandwidth usages.

---

Now assume that the attacker spoofs source IP addresses such that the customer no longer receivers RST packets (in response to SYN-ACK packets). This will cause the customer to send 4 SYN-ACK packets in total before the handshake time outs. **Calculate** the outgoing bandwidth of the customer consumed by the attack.

YOUR ANSWER HERE

##### d) (1 point)

Well, the equipment to handle that kind of bandwidth, along with the ISP cost, definitely is too expensive for the customer, who decides to use a combined Content Delivery Network / Web Application Firewall solution instead. Still, one question still lingers in the room:

---

Is it possible for the NoodleFort equipment to detect and blocked spoofed source IP addresses?

YOUR ANSWER HERE

## 2. Firewalls (8 points)

While it is sad that the deal for the improved network hardware did not close, the next task comes in too quick for you to ponder about it any longer: NoodleFort has decided to completely revamp a lot of its software architecture, including a completely new firewall implementation.
The software architect's gameplan is to first start with a simple packet filter, then step up to a bidirectional firewall, and then upgrade to a stateful firewall.

To test the implementation along the way, NoodleFort already has created a packet capture to run the firewall on.

*Hint: Use the provided cheatsheet as reference for scapy.*

In [ ]:
from scapy.all import *

# read pcap
packets = rdpcap("./packets.pcapng")

### Task 2.1 (2 points)

The first task is to implement a simple packet filter. The filter should accept all DNS packets sent to the default DNS port using UDP (we will ignore DNS over TCP / TLS / HTTP packets here).

**Implement** the function `toy_dns_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]`. As argument, this function gets the packet list `packets` that was loaded prior. The function should return a tuple of two different packet lists `(accept_list, deny_list)` where `accept_list` contains a list of all packets accepted by your firewall using the specified rule above and `deny_list` contains a list of all denied packets.

The type `Packet` here and in the following is meant as `scapy.packet.Packet`.

In [ ]:
def toy_dns_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]:
    accept: list[Packet] = []
    deny: list[Packet] = []
    # YOUR CODE HERE
    raise NotImplementedError()
    return accept, deny

In [ ]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [ ]:
# This test just checks the output of your solution

res = toy_dns_firewall(packets)

assert type(res) == tuple, "Your function does not return a tuple!"
assert len(res) == 2, "The returned tuple does not contain exactly two elements!"
assert type(res[0]) == list, "The first element in your tuple is not a list!"
assert type(res[1]) == list, "The second element in your tuple is not a list!"
assert len(res[0]) > 100 and len (res[0]) < 1100, "Something is strange. Check if your rule works properly."
assert len(res[1]) > 100 and len (res[1]) < 1100, "Something is strange. Check if your rule works properly."
assert len(res[0]) + len(res[1]) == 1241, "You lost some of the packets during filtering!"

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.2 (2 points)

Now, we have a problem. We allow DNS messages sent through our firewall to the DNS server, but we do not allow the answers sent from this server to some client behind our firewall. We want to accept all packets that have the default DNS port as destination or source port and are running over UDP.

**Implement** the function `toy_dns_bidirect_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]`. The arguments and the return values are the same as in the task before, only your firewall policy should change. The function gets a packet list `packets` and returns a tuple `(accept_list, deny_list)` with the list of accepted or denied packets.

In [ ]:
def toy_dns_bidirect_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]:
    accept: list[Packet] = []
    deny: list[Packet] = []
    # YOUR CODE HERE
    raise NotImplementedError()
    return accept, deny

In [ ]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [ ]:
# This test just checks the output of your solution

res = toy_dns_bidirect_firewall(packets)

assert type(res) == tuple, "Your function does not return a tuple!"
assert len(res) == 2, "The returned tuple does not contain exactly two elements!"
assert type(res[0]) == list, "The first element in your tuple is not a list!"
assert type(res[1]) == list, "The second element in your tuple is not a list!"
assert len(res[0]) > 100 and len (res[0]) < 1100, "Something is strange. Check if your rule works properly."
assert len(res[1]) > 100 and len (res[1]) < 1100, "Something is strange. Check if your rule works properly."
assert len(res[0]) + len(res[1]) == 1241, "You lost some of the packets during filtering!"

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.3 (4 points)

Even we improved our firewall implementation in the last subtask, it is still lacking a crucial detail. Usually, Firewalls nowadays have a stateful filter mechanism. This ensures that answers to packets are only allowed if there was an initial packet sent from us. To enable this functionality, we have to keep track of open connections.

The firewall policy is now to accept any packet that is a response to an earlier packet that came from the address `192.168.7.1` and deny any packet that is not a response to a packet sent from that IP address. A packet *R* is considered to be a response to a packet *P* if the source of *P* is now the destination of *R* and the destination of *P* is now the source of *R*. The source and destination of a packet contains the IP and the port number in our case. You do not need to deal with 'forgetting' connections. Keep in mind that any outgoing packet (i.e. sent from `192.168.7.1`) has to be accepted as well. **We also only care about TCP connections and deny all UDP packets.**

**Implement** the the firewall policy in the function `toy_state_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]` as described above. The arguments and the return values are the same as in the task before, only your firewall policy should change.

*Hint: you have to keep track of your initiated connections and the respective communication endpoints.*

In [ ]:
def toy_state_firewall(packets: list[Packet]) -> tuple[list[Packet], list[Packet]]:
    accept: list[Packet] = []
    deny: list[Packet] = []
    # YOUR CODE HERE
    raise NotImplementedError()
    return accept, deny

In [ ]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

accept, deny = toy_state_firewall(packets)
print(len(accept))
print(len(deny))

In [ ]:
# This test just checks the output of your solution

res = toy_state_firewall(packets)

assert type(res) == tuple, "Your function does not return a tuple!"
assert len(res) == 2, "The returned tuple does not contain exactly two elements!"
assert type(res[0]) == list, "The first element in your tuple is not a list!"
assert type(res[1]) == list, "The second element in your tuple is not a list!"
assert len(res[0]) < 100, "Something is strange. Check if your rule works properly."
assert len(res[1]) > 100, "Something is strange. Check if your rule works properly."
assert len(res[0]) + len(res[1]) == 1241, "You lost some of the packets during filtering!"

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 3. Intruders in a Haystack (4.5 points)

With the great success of your firewall implementation, you quickly rise through the internal ranks and quickly become responsible for bigger decisions. One of these decisions concerns the intrusion detection system integrated into the next-generation firewall. You need to decide how confident the IDS must be for it to raise an alarm.

Your subordinate, Lerry, has already prepared two thresholds: The first threshold is rather trigger happy and thus is able to catch all attacks the system was tested against, but also has a 1% chance to cause an alarm on a benign packet. The alternative is to be absolutely certain that an attack is going, leading to the system having only an 80% chance of detecting malicious packets, which then reduces the alarm chance on benign packets to $10^{-8}$.

With his background in technical sales, Lerry strongly advocates for the first confidence threshold: "Imagine being able to tell customers that we can detect all attacks, all of the time. I can already smell the provision hitting my bank account!". In fact, you suspect that Lerry deliberatly chose the seconds confidence threshold to detect as few attacks as possible to easier sell you the first threshold. This makes you cautious, and want to look at both confidence thresholds from all angles.

---

#### a) (2 points)

**Calculate** the probability $P(benign | alarm)$ for both confidence thresholds as probability between 0 and 1, assuming that the probability of any given packet to be malicious is $10^{-6}$.

*Hint: You need at least two decimal places of precision, i.e., the first two digits after the decimal point need to be correct.*

In [ ]:
def return_false_alarm_rate_of_first_threshold() -> float:
    # P(malicious) = 10^-6
    # P(alarm | malicious) = 1
    # P(alarm | benign) = 0.01
    # You can hard-code the false alarm rate.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def return_false_alarm_rate_of_second_threshold() -> float:
    # P(malicious) = 10^-6
    # P(alarm | malicious) = 0.8
    # P(alarm | benign) = 10^-8
    # You can hard-code the false alarm rate.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
### PLAYGROUND

print(f"False alarm rate of the first threshold: {return_false_alarm_rate_of_first_threshold()}")
print(f"False alarm rate of the second threshold: {return_false_alarm_rate_of_second_threshold()}")

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

#### b) (2 points)

Given these false alarm rates, **choose** which threshold to implement in the next-generation firewall. **Argue** why your chosen threshold is more useful than the other one in practice, i.e., also explicitely argue against the other threshold.

YOUR ANSWER HERE

#### c) (0.5 points)

How is the fallacy called that leads to false alarm rates often being much higher than intuitively guessed (by most humans)?

YOUR ANSWER HERE

## 4. Access granted? (2 points)

Finally, your coworker Lerry asks you to look over his work. He had to configure a Unix filesystem for a customer with some pretty complex requirements, and wants to make sure he did not screw up.
His current configuration looks this:

```
--w-r---w- david   groupB file1
-r--r----x bob     groupA file2
----r----- charlie groupD file3
drwxrwx--- alice   groupB dir1
-rw--w-r-- alice   groupF dir1/file4

groupA = {alice, david}
groupB = {bob, charlie}
groupC = {charlie}
groupD = {david}
groupE = {bob, david}
groupF = {alice, bob}
```

One of the customer's requirements were that Bob must only be able to access `file2`.

---

**List** all files which the user `bob` can write to directly or indirectly (after legal permission changes done by `bob`). 

*In the exam, we would typically ask to also briefly comment on the reason why the user has write access to that file for each file you list.*

In [ ]:
def return_writeable_files() -> list[str]:
    # You can use the following as template for your solution.
    # The order of the files does not matter.
    # return [
    #     #"file1",
    #     #"file2",
    #     #"file3",
    #     #"dir1/file4"
    # ]
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 5. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

YOUR ANSWER HERE